In [1]:
import time, gym, sys
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from scipy.stats import geom
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt
import homework.hw1.load_policy as load_policy
import homework.hw1.tf_util as tf_util
import tensorflow as tf
from copy import deepcopy as dc
import numba

sigma = 0.1

dim = 17
dim_act = 6
obs_min = np.array([-0.25,-0.8 ,-0.7 ,-0.7,-0.65,-0.75,-0.95,-0.6 ,-1.5,-3.1,-7.1,-20,-24,-27,-27,-30,-20])
obs_max = np.array([ 0.4 , 1.65, 0.95, 0.9, 0.95, 0.95, 1.1 , 0.75, 8  , 3.4, 7  , 19, 25, 22, 25, 32, 26])

/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/torbensell/anaconda3/lib/python3.6/site-packages/tensorflow/

In [2]:
"""

initialise KL

"""

alpha = 2
def eigenvalues_1():
    ev_1 = np.zeros(2*N_trunc-2)
    for i in range(1,N_trunc):
        c_i = 1/((i+1)**2+(i+1)**2)**alpha
        ev_1[2*i-2] = c_i
        ev_1[2*i-1] = c_i
    ev_1 = 4*ev_1
    return ev_1**2

def eigenvalues_2():
    ev_2 = np.zeros(4*N_trunc**2-4)
    for i in range(1,N_trunc):
        for j in range(1,N_trunc):
            c_ij = 1/((i+1)**2+(j+1)**2)**alpha
            ev_2[4*(i+N_trunc*j)-4] = c_ij
            ev_2[4*(i+N_trunc*j)-3] = c_ij
            ev_2[4*(i+N_trunc*j)-2] = c_ij
            ev_2[4*(i+N_trunc*j)-1] = c_ij
    ev_2 = 4*ev_2
    return ev_2**2

def C_init():
    C = np.zeros((4*N_trunc**2-4,dim,dim))
    C_root = np.zeros((4*N_trunc**2-4,dim,dim))
    
    for j in range(dim):
        for i_1 in range(1,N_trunc):
            C[2*i_1-2,j,0] = ev_1[2*i_1-2]
            C[2*i_1-1,j,0] = ev_1[2*i_1-1]
            
    for j in range(dim):
        for k in range(dim):
            if k>j:
                for i_1 in range(1,N_trunc):
                    for i_2 in range(1,N_trunc):
                        C[4*(i_1+N_trunc*i_2)-4,j,k] = ev_2[4*(i_1+N_trunc*i_2)-4]
                        C[4*(i_1+N_trunc*i_2)-3,j,k] = ev_2[4*(i_1+N_trunc*i_2)-3]
                        C[4*(i_1+N_trunc*i_2)-2,j,k] = ev_2[4*(i_1+N_trunc*i_2)-2]
                        C[4*(i_1+N_trunc*i_2)-1,j,k] = ev_2[4*(i_1+N_trunc*i_2)-1]
    C_root = C**(1/2)
    return C,C_root

In [3]:
"""

INITIALISE ACTION AND OBSERVATION SPACE

"""

actions = []
actions_matrix = np.load('HC_actions.npy')
N_a = 8
for i in range(N_a):
    actions.append(actions_matrix[i,:])

In [4]:
"""

FUNCTIONS - PRIOR

"""

def sample_prior(c=1):
    xi = np.zeros((4*N_trunc**2-4,dim,dim))
    
    for j in range(dim):
        for i_1 in range(1,N_trunc):
            xi[2*i_1-2,j,0] = C_root[2*i_1-2,j,0]*np.random.normal()
            xi[2*i_1-1,j,0] = C_root[2*i_1-1,j,0]*np.random.normal()
            
    for j in range(dim):
        for k in range(dim):
            if k>j:
                for i_1 in range(1,N_trunc):
                    for i_2 in range(1,N_trunc):
                        xi[4*(i_1+N_trunc*i_2)-4,j,k] = C_root[4*(i_1+N_trunc*i_2)-4,j,k]*np.random.normal()
                        xi[4*(i_1+N_trunc*i_2)-3,j,k] = C_root[4*(i_1+N_trunc*i_2)-3,j,k]*np.random.normal()
                        xi[4*(i_1+N_trunc*i_2)-2,j,k] = C_root[4*(i_1+N_trunc*i_2)-2,j,k]*np.random.normal()
                        xi[4*(i_1+N_trunc*i_2)-1,j,k] = C_root[4*(i_1+N_trunc*i_2)-1,j,k]*np.random.normal()
    return xi*c

def logprior(xi):
    logprior = 0
    
    for j in range(dim):
        for i in range(1,N_trunc):
            logprior += np.log(norm._pdf(xi[2*i-2,j,0]/C_root[2*i-2,j,0]))
            logprior += np.log(norm._pdf(xi[2*i-1,j,0]/C_root[2*i-1,j,0]))
    
    for j in range(dim):
        for k in range(dim):
            if k>j:
                for i_1 in range(1,N_trunc):
                    for i_2 in range(1,N_trunc):
                        logprior += np.log(norm._pdf(xi[4*(i_1+N_trunc*i_2)-4,j,k]/C_root[4*(i_1+N_trunc*i_2)-4,j,k]))
                        logprior += np.log(norm._pdf(xi[4*(i_1+N_trunc*i_2)-3,j,k]/C_root[4*(i_1+N_trunc*i_2)-3,j,k]))
                        logprior += np.log(norm._pdf(xi[4*(i_1+N_trunc*i_2)-2,j,k]/C_root[4*(i_1+N_trunc*i_2)-2,j,k]))
                        logprior += np.log(norm._pdf(xi[4*(i_1+N_trunc*i_2)-1,j,k]/C_root[4*(i_1+N_trunc*i_2)-1,j,k]))
                    
    return logprior

In [5]:
"""

FUNCTIONS - LIKELIHOOD 

"""

''' Function to integrate within likelihood '''
def f(x,v_a):
    value = norm._pdf((x-v_a[0])/sigma)/sigma
    for i in range(1,N_a):
        value = value*norm._cdf((x-v_a[i])/sigma)
    return value

def likelihood(xi,pair):
    poss = pair[0:8]
    vels = pair[8:17]
    action = int(pair[17])
    poss = np.insert(poss,0,0)
    
    ''' Check where all actions would take us '''
    x = np.zeros((N_a,dim))
    v = np.zeros(N_a)
    for j in range(N_a):
        env.set_state(poss, vels)
        x[j,:] = env.step(actions[j])[0]
        v[j] = u(xi,x[j,:])
        
    ''' Sort v such that the first entry is the taken action '''
    v_a = np.zeros(N_a)
    if action!=0:
        v_a[0] = v[action]
        v_a[1:action+1] = v[0:action]
        v_a[action+1:] = v[action+1:]
    else:
        v_a = v
            
    ''' Integrate over pdf of chosen action and cdf of other actions '''
    lklhd = quad(f,v_a[0]-3*sigma,v_a[0]+3*sigma,args=v_a,limit=500)[0]
    return lklhd

def loglikelihood(xi,data):
    loglikelihood = 0
    for j in range(data.shape[0]):
        loglikelihood += np.log(likelihood(xi,data[j,:]))
    return loglikelihood

''' partial derivative du/dxi '''
@numba.njit()
def diff_u(x,shape):    
    diff = np.zeros(shape)
    for j in range(dim):
        
        ''' 1D terms '''
        j_eval = np.pi*(x[j]-obs_min[j])/(obs_max[j]-obs_min[j])
        for i in range(1,N_trunc):
            c_j = 2/(obs_max[j]-obs_min[j])
            diff[2*i-2,j,0] += c_j*np.cos(i*j_eval)
            diff[2*i-1,j,0] += c_j*np.sin(i*j_eval)
            
        ''' 2D terms '''   
        for k in range(dim):
            if k>j:
                j_eval = np.pi*(x[j]-obs_min[j])/(obs_max[j]-obs_min[j])
                k_eval = np.pi*(x[k]-obs_min[k])/(obs_max[k]-obs_min[k])
                for i_1 in range(1,N_trunc):
                    for i_2 in range(1,N_trunc):
                        c_jk = 4/(obs_max[j]-obs_min[j])/(obs_max[k]-obs_min[k])
                        diff[4*(i_1+N_trunc*i_2)-4,j,k] += c_jk*np.cos(i_1*j_eval)*np.cos(i_2*k_eval)
                        diff[4*(i_1+N_trunc*i_2)-3,j,k] += c_jk*np.cos(i_1*j_eval)*np.sin(i_2*k_eval)
                        diff[4*(i_1+N_trunc*i_2)-2,j,k] += c_jk*np.sin(i_1*j_eval)*np.cos(i_2*k_eval)
                        diff[4*(i_1+N_trunc*i_2)-1,j,k] += c_jk*np.sin(i_1*j_eval)*np.sin(i_2*k_eval)
    return diff

''' function which is integrated in likelihood gradient '''
def f_grad(t,args):
    v_a = args[0]
    j = args[1]
    if j==0:
        value = (t-v_a[0])/(sigma**2)*norm._pdf((t-v_a[0])/sigma)/sigma
    else:
        value = -norm._pdf((v_a[0]-v_a[j])/(np.sqrt(2)*sigma))/(np.sqrt(2)*sigma)*norm._pdf((t-(v_a[0]+v_a[j])/2)/(sigma/np.sqrt(2)))/(sigma/np.sqrt(2))
    for i in range(1,N_a):
        if i!=j:
            value = value*norm._cdf((t-v_a[i])/sigma)
    return value

''' partial derivative dl/dv '''
def grad_ll(v_a,j):
    if j==0:
        return quad(f_grad,v_a[0]-3*sigma,v_a[0]+3*sigma,args=[v_a,j],limit=500)[0]
    else:
        return quad(f_grad,(v_a[0]+v_a[j])/2-3*sigma,(v_a[0]+v_a[j])/2+3*sigma,args=[v_a,j],limit=500)[0]

    
def diff_ll(xi,data):
    diff = np.zeros(xi.shape)
        
    x = np.zeros((N_a,dim))
    v = np.zeros(N_a)
    x_a = np.zeros((N_a,dim))
    v_a = np.zeros(N_a)
    grad_a = np.zeros(N_a)
    data_state = np.zeros(dim)
    
    ''' Iterate through all or a subset of the data points, and compute the respective gradients '''
    if stochastic_gradients and unadjusted:
        range_i = random.sample(range(data.shape[0]),10)
    else:
        range_i = range(data.shape[0])
    for i in range_i:
        lh = likelihood(xi,data[i,:])
        data_poss = data[i,0:8]
        data_vels = data[i,8:17]
        data_poss = np.insert(data_poss,0,0)
        data_action = int(data[i,-1])
        
        for j in range(N_a):
            env.set_state(data_poss, data_vels)
            x[j,:] = env.step(j)[0]
            v[j] = u(xi,x[j,:])
        
        # sort v and x such that the first entry is the taken action
        if data_action!=0:
            v_a[0] = v[data_action]
            v_a[1:data_action+1] = v[0:data_action]
            v_a[data_action+1:] = v[data_action+1:]
            x_a[0,:] = x[data_action,:]
            x_a[1:data_action+1,:] = x[0:data_action,:]
            x_a[data_action+1:,:] = x[data_action+1:,:]
        else:
            v_a = v
            x_a = x
            
        for j in range(N_a):
            grad_a[j] = grad_ll(v_a,j)/lh
        mean_grad = np.mean(grad_a)
        grad_a -= mean_grad
            
        for j in range(N_a):
            diff += grad_a[j]*diff_u(x_a[j,:],xi.shape)
            
    return diff

In [6]:
"""

FUNCTIONS - VALUE FUNCTION AND POLICIES

"""

''' u(xi,x), which evaluates the function u with coefficients xi at x, fast version '''
@numba.njit()
def u(xi,x):
    u_sum = 0
    for j in range(dim):
        
        ''' 1D terms '''
        j_eval = np.pi*(x[j]-obs_min[j])/(obs_max[j]-obs_min[j])
        for i in range(1,N_trunc):
            c_j = 2/(obs_max[j]-obs_min[j])
            u_sum += c_j*xi[2*i-2,j,0]*np.cos(i*j_eval)
            u_sum += c_j*xi[2*i-1,j,0]*np.sin(i*j_eval)
            
        ''' 2D terms '''   
        for k in range(dim):
            if k>j:
                j_eval = np.pi*(x[j]-obs_min[j])/(obs_max[j]-obs_min[j])
                k_eval = np.pi*(x[k]-obs_min[k])/(obs_max[k]-obs_min[k])
                for i_1 in range(1,N_trunc):
                    for i_2 in range(1,N_trunc):
                        c_jk = 4/(obs_max[j]-obs_min[j])/(obs_max[k]-obs_min[k])
                        u_sum += c_jk*xi[4*(i_1+N_trunc*i_2)-4,j,k]*np.cos(i_1*j_eval)*np.cos(i_2*k_eval)
                        u_sum += c_jk*xi[4*(i_1+N_trunc*i_2)-3,j,k]*np.cos(i_1*j_eval)*np.sin(i_2*k_eval)
                        u_sum += c_jk*xi[4*(i_1+N_trunc*i_2)-2,j,k]*np.sin(i_1*j_eval)*np.cos(i_2*k_eval)
                        u_sum += c_jk*xi[4*(i_1+N_trunc*i_2)-1,j,k]*np.sin(i_1*j_eval)*np.sin(i_2*k_eval)
    return u_sum
    
''' Policy from berkeley course '''
def policy(obs):
    return policy_fn(obs[None,:])
print('loading and building expert policy')
policy_fn = load_policy.load_policy('/Users/torbensell/Dropbox (Cambridge University)/UNI/CAM/PhD/Programme/BIRL_NN_pCN/homework/hw1/experts/HalfCheetah-v2.pkl')
print('loaded and built')

loading and building expert policy
obs (1, 17) (1, 17)
loaded and built


In [7]:
"""

FUNCTIONS - ANALYTICS

"""

''' Progress bar to know how much longer one has to wait '''
def progressBar(t,value, t_max, acceptances, bar_length=40):
    percent = float(t) / t_max
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\rIteration: {0}    Acceptance ratio: {1}    Percent: [{2}] {3}%  ".format(value,round(acceptances/value,3),arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()   
    
''' Plotting a value function '''    
def func_plot(xi,name):
    x = np.arange(obs_min[0],obs_max[0],0.02)
    y = np.arange(obs_min[1],obs_max[1],0.05)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            x=np.zeros(dim)
            x[0]=X[i,j]
            x[1]=Y[i,j]
            Z[i,j] = u(xi,x)
            
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdBu,linewidth=0, antialiased=False)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')
    ax.set_zlabel('z-axis')
    ax.view_init(elev=25, azim=-120)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)
    
def trajectory_plot(xi,name):
    x = np.arange(len(xi))
    fig = plt.figure()
    plt.plot(x,xi)
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)
    
def autocorr(x,lags):
    mean=np.mean(x)
    var=np.var(x)
    xp=x-mean
    corr=[1. if l==0 else np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    return np.array(corr)

''' Calculate the Effective Sample Size, assumes algorithm already burned in '''
def ESS(logposterior,name):
    fig, ax = plt.subplots()
    N = len(logposterior)
    ax.stem(autocorr(logposterior, range(int(N*0.1))),use_line_collection=True) 
    ESS = N/(1+2*sum(autocorr(logposterior, range(int(N*0.1)))))
    print('\nEffective Sample Size:', round(ESS))
    print('Samples required to generate 1 independent sample:', round(N/ESS,2))
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)    

In [8]:
"""

Uncertainty Quantification Initialisation

"""

x_test = np.load('HC_x_test.npy')
v_test = []
for i in range(N_a):
    v_test.append([])
    
x_test_100 = np.load('HC_x_test_100.npy')
a_test_100 = np.load('HC_a_test_100.npy')
optimal_choice = 0
not_optimal_choice = 0
v_all = np.zeros((N_a,100))

def test_value_fn(xi):
    for i in range(N_a):
        v = u(xi,x_test[i,:])
        v_test[i].append(v)
        if i>0:
            v_test[i][-1]=v_test[i][-1]-v_test[0][-1]
    v_test[0][-1] = 0
    
    global optimal_choice
    global not_optimal_choice
    v = np.zeros(N_a)
    for j in range(100):
        for i in range(N_a):
            v[i] = u(xi,x_test_100[i,:,j])
            v_all[i,j] += v[i]
        if a_test_100[j]==np.argmax(v):
            optimal_choice += 1
        else:
            not_optimal_choice += 1
            
        
def boxplot_value_fn():
    global v_test
    global optimal_choice
    global not_optimal_choice
    
    fig, ax = plt.subplots()
    ax.set_title('UQ of relative value function evaluation')
    reward_test = np.load('HC_reward_test.npy')
    tags = []
    for i in range(N_a):
        tags.append(str(round(reward_test[i],2)))
    ax.set_xticklabels(tags, rotation=90)
    ax.boxplot(v_test)
    fig.savefig('figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_UQ.pdf', dpi=300)
    plt.close(fig)
    
    fig, ax = plt.subplots()
    ax.set_title('UQ of relative value function evaluation')
    ax.boxplot(v_test)
    fig.savefig('figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_UQ_v2.pdf', dpi=300)
    plt.close(fig)
    
    v_test = []
    for i in range(N_a):
        v_test.append([])
        
    print('Samples, Correct choices: ',optimal_choice/(optimal_choice+not_optimal_choice))
    print('Samples, Wrong choices:   ',not_optimal_choice/(optimal_choice+not_optimal_choice))
    
    ''' Mean values '''
    optimal_choice = 0
    not_optimal_choice = 0
    
    for j in range(100):
        for i in range(N_a): 
            if a_test_100[j]==np.argmax(v_all[:,j]):
                optimal_choice += 1
            else:
                not_optimal_choice += 1
                
    print('Mean,    Correct choices: ',optimal_choice/(optimal_choice+not_optimal_choice))
    print('Mean,    Wrong choices:   ',not_optimal_choice/(optimal_choice+not_optimal_choice))

In [9]:
"""

FUNCTIONS - MCMC (pCN/pCNL)

"""
    
def acceptance_prop(xi_u, xi_v,data,ll_u,diff_u=False):
    accept_prop = -ll_u
    ll_v = 0
        
    for j in range(data.shape[0]):
        lh = likelihood(xi_v,data[j,:])
        ll_v += np.log(lh)
    accept_prop += ll_v
        
    ''' Only pCNL (has extra terms) '''
    if method=='pCNL':
        accept_prop += -np.sum((xi_v-xi_u)*diff_u)/2 - delta*np.sum((xi_u+xi_v)*diff_u)/4 + delta*np.linalg.norm(diff_u*C_root)**2/4
        diff_v = diff_ll(xi_v,data)
        accept_prop -= -np.sum((xi_u-xi_v)*diff_v)/2 - delta*np.sum((xi_v+xi_u)*diff_v)/4 + delta*np.linalg.norm(diff_v*C_root)**2/4
        return min(1, np.exp(accept_prop)),ll_v,diff_v
    else:
        return min(1, np.exp(accept_prop)),ll_v
    
def propose(xi,diff=False):
    if method=='pCNL':
        proposal = ((2-delta)*xi + 2*delta*C*diff + np.sqrt(8*delta)*sample_prior())/(2+delta)     
    else:
        proposal = np.sqrt(1-beta*beta)*xi+beta*sample_prior()
    return proposal

def MCMC(xi,N_data,data,max_time):   
    print('\nMCMC algorithm ('+method + ', N_trunc=' + str(N_trunc) + ', N_data=' + str(N_data) + ', ' + str(max_time) + ' seconds) was started: ' + str(time.ctime()))
      
    acc_ratio = 0
    logposterior = []
    logp = []
    logl = []
    
    ''' Initialise likelihood and gradient '''  
    ll = loglikelihood(xi,data)
    print('Initial loglikelihood: ',ll)
    if method=='pCNL' or method=='CNL':
        diff = diff_ll(xi,data)
    
    ''' Run MCMC '''
    start = time.time() 
    j = 0
    it = 0
    while(time.time()-start<max_time):
        
        ''' Propose and calculate acceptance probability '''
        if method=='pCNL' or method=='CNL':
            xi_proposal = propose(xi,diff)  
            a,ll_proposal,diff_proposal = acceptance_prop(xi,xi_proposal,data,ll,diff)
        else:
            xi_proposal = propose(xi)  
            a,ll_proposal = acceptance_prop(xi,xi_proposal,data,ll)
        
        ''' Accept or reject proposal '''
        uni = np.random.uniform()
        if uni < a or unadjusted:
            if method=='pCNL' or method=='CNL':
                diff = diff_proposal
            xi = xi_proposal    
            ll = ll_proposal
            acc_ratio = acc_ratio + 1

        ''' prior, likelihood, and posterior traceplots are appended '''
        lp = logprior(xi)
        logposterior.append(lp+ll)
        logp.append(lp)
        logl.append(ll)
        
        if prior_compare and j%10==0:
            ''' store value function evaluations for uncertainty estimates '''
            test_value_fn(xi)
        elif policy_compare and (time.time()-start)>it*t_max/1000 and it<1000:
            ''' store sample for future use '''
            np.save('np_saved/HC/samples_policy_learning/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_sampleNo'+str(it)+'.npy',xi)
            it += 1
        
        if method=='pCNL' or method=='CNL':
            progressBar(time.time()-start,j+1,max_time,acc_ratio)
        elif (j+1)%100==0:
            progressBar(time.time()-start,j+1,max_time,acc_ratio)
        j+=1
        
    progressBar(max_time,j,max_time,acc_ratio)
    
    acc_ratio = acc_ratio/(j)
    print('\nMCMC algorithm terminated: ' + str(time.ctime()) + '. \nRuntime = ' + str(time.time()-start))
    print('Final loglikelihood: ',ll)
    print('Acceptance ratio is ',acc_ratio)
    
    np.save('np_saved/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample.npy',xi)
    
    trajectory_plot(logposterior[1:],'figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_logposterior')
    trajectory_plot(logp[1:],'figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_logprior')
    trajectory_plot(logl[1:],'figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_loglikelihood')
    if prior_compare:
        boxplot_value_fn()

    func_plot(xi,'figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample')
    
    ESS(logposterior,'figs/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_autocorr')    

# MAIN PROGRAMMES

In [10]:
"""

MAIN PROGRAMME 1 - compare NN prior to KL prior (large number of parameters)

"""    

prior_compare = True
policy_compare = False  

# set maximal runtime
t_max = 3600*24

optimal_choice = 0
not_optimal_choice = 0

N_trunc = 10 # make sure to restart the kernel and recompile the numba compiled code
ev_1 = eigenvalues_1()
ev_2 = eigenvalues_2()
C,C_root = C_init()

# Sample from the prior to see what a sample looks like
xi = sample_prior()
func_plot(xi,'figs/HC/KL_'+str(N_trunc)+'_a_prior_sample')

env = gym.make('HalfCheetah-v2')
env.unwrapped
env.reset()
    
try:
    data = np.load('HC_data.npy')
except FileNotFoundError:
    print('\nNo data found - create data!\n')
N_data = 100
    
''' run pCN '''
method = 'pCN'
stochastic_gradients = False # if true then unadjusted needs to be true too
unadjusted = False
beta = 1/16
try:
    xi = np.load('np_saved/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample.npy')
except FileNotFoundError:
    print('\nStarting from close to 0')
    xi = sample_prior(beta)
MCMC(xi,N_data,data[50:50+N_data,:],t_max) 

env.close()


MCMC algorithm (pCN, N_trunc=10, N_data=100, 86400 seconds) was started: Sun Jul 12 21:18:55 2020
Initial loglikelihood:  -68.29045368692636
Iteration: 140104    Acceptance ratio: 0.238    Percent: [--------------------------------------->] 100%  
MCMC algorithm terminated: Mon Jul 13 21:18:56 2020. 
Runtime = 86400.2601120472
Final loglikelihood:  -78.37091158961736
Acceptance ratio is  0.23843002341118028
Samples, Correct choices:  0.2012375990293341
Samples, Wrong choices:    0.7987624009706659
Mean,    Correct choices:  0.25
Mean,    Wrong choices:    0.75

Effective Sample Size: 211.0
Samples required to generate 1 independent sample: 664.91


In [10]:
"""

MAIN PROGRAMME 2 - LEARN policy, and store samples for future use (small number of parameters)

"""    

prior_compare = False
policy_compare = True   

# set maximal runtime
t_max = 3600*24

N_trunc = 5 # make sure to restart the kernel and recompile the numba compiled code
ev_1 = eigenvalues_1()
ev_2 = eigenvalues_2()
C,C_root = C_init()

# Sample from the prior to see what a sample looks like
xi = sample_prior()
func_plot(xi,'figs/HC/KL_'+str(N_trunc)+'_a_prior_sample')

env = gym.make('HalfCheetah-v2')
env.unwrapped
env.reset()
    
try:
    data = np.load('HC_data.npy')
except FileNotFoundError:
    print('\nNo data found - create data!\n')
N_data = 100
    
    
''' run pCN '''
method = 'pCN'
stochastic_gradients = False # if true then unadjusted needs to be true too
unadjusted = False
beta = 1/16
try:
    xi = np.load('np_saved/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample.npy')
except FileNotFoundError:
    print('\nStarting from close to 0')
    xi = sample_prior(beta)
MCMC(xi,N_data,data[50:50+N_data,:],t_max) 

''' run pCNL '''
method = 'pCNL'
stochastic_gradients = False # if true then unadjusted needs to be true too
unadjusted = False
delta = 1/5800
try:
    xi = np.load('np_saved/HC/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample.npy')
except FileNotFoundError:
    print('\nStarting from close to 0')
    xi = sample_prior(delta)
MCMC(xi,N_data,data[50:50+N_data,:],t_max) 


env.close()


MCMC algorithm (pCNL, N_trunc=5, N_data=100, 86400 seconds) was started: Tue Jul 21 09:04:37 2020
Initial loglikelihood:  -70.65911151775056
Iteration: 33242    Acceptance ratio: 0.592    Percent: [--------------------------------------->] 100%  
MCMC algorithm terminated: Wed Jul 22 09:04:41 2020. 
Runtime = 86401.55527281761
Final loglikelihood:  -79.06841893579815
Acceptance ratio is  0.5920221406654232

Effective Sample Size: 6.0
Samples required to generate 1 independent sample: 5901.45
